<table class="table table-bordered">
    <tr>
        <th style="width:250px"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png?sfvrsn=764583a6_0' style="width: 100%; height: 125px; "></th>
        <th style="text-align:center;"><h1>Distributed Data Pipelines</h1><h2>Assignment 1 </h2><h3>Diploma in Data Science</h3></th>
    </tr>
</table>

Learning Objectives:
- Design PySpark Based Machine Learning
- Execute PySpark Syntax Correctly
- Evaluate and Select Final Model based on Metrics

You will be **graded on the use of PySpark**, so usage of **Pandas itself should be avoided as much as possible**, especially if a particular native method or function is already available in PySpark. **Penalties will be imposed in such cases.**

# Distributed Data Pipelines ASG1, Ambrish Krishna Muralitharan, S10223486G<a id="top"></a>

## Table of Contents


### 7.  [Report](#part7) 
   - [7.1 Problem Statement Formulation](#part7.1)
   - [7.2 Exploratory Data Analysis and Data Cleansing](#part7.2)
   - [7.3 Data Wrangling & Transformation](#part7.3)
   - [4.4 Machine Learning Modelling](#part7.4)
   - [4.5 Model Evaluation & Selection](#part7.5)
   - [4.6 Summary & Further Improvements](#part7.6)

In [32]:
# import the packages
#Basic
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ASG1').getOrCreate()

#Functions
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.functions import desc, asc, udf

#Types
from pyspark.sql.types import StringType

#Matplotlib
from matplotlib import pyplot as plt
from pyspark.sql import *

#Imputation
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import Bucketizer

#Categorical Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer

#Feature Scaling
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

#Machine Learning
from pyspark.ml.regression import LinearRegression

### Step 1: Problem Statement Formulation

In [33]:
# load and explore data
df=spark.read.csv('./data/sg_flat_prices_mod.csv', header=True, inferSchema=True) # inferSchema auto detects data type

In [34]:
df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- floor_area_sqm: double (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)



In [35]:
df.show(truncate=False)

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|year|month|town      |flat_type|block|street_name      |storey_range|floor_area_sqm|flat_model    |lease_commence_date|remaining_lease|resale_price|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+
|2017|1    |ANG MO KIO|2 ROOM   |406  |ANG MO KIO AVE 10|10 TO 12    |44.0          |Improved      |1979               |736            |232000.0    |
|2017|1    |ANG MO KIO|3 ROOM   |108  |ANG MO KIO AVE 4 |01 TO 03    |67.0          |New Generation|1978               |727            |250000.0    |
|2017|1    |ANG MO KIO|3 ROOM   |602  |ANG MO KIO AVE 5 |01 TO 03    |67.0          |New Generation|1980               |749            |262000.0    |
|2017|1    |ANG MO KIO|3 ROOM   |465  |ANG MO KIO AVE 10|04 TO 06    |68.0          |New Generation|

In [36]:
# Shape of the dataset
print('Shape of the dataset: ', (df.count(), len(df.columns)))

Shape of the dataset:  (64247, 12)


In [37]:
df.describe('year', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range').show()

+-------+------------------+------------------+----------+----------------+------------------+------------+------------+
|summary|              year|             month|      town|       flat_type|             block| street_name|storey_range|
+-------+------------------+------------------+----------+----------------+------------------+------------+------------+
|  count|             64247|             64247|     64247|           64247|             64247|       64247|       64247|
|   mean|2018.0262424704656| 6.779133656046197|      null|            null| 349.4635640052788|        null|        null|
| stddev|0.8146939469668695|3.2635673352950514|      null|            null|254.80560486394563|        null|        null|
|    min|              2017|                 1|ANG MO KIO|          1 ROOM|                 1|ADMIRALTY DR|    01 TO 03|
|    max|              2019|                12|    YISHUN|MULTI-GENERATION|                9B|     ZION RD|    49 TO 51|
+-------+------------------+----

In [38]:
df.describe('floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price').show()

+-------+-----------------+-------------+-------------------+------------------+------------------+
|summary|   floor_area_sqm|   flat_model|lease_commence_date|   remaining_lease|      resale_price|
+-------+-----------------+-------------+-------------------+------------------+------------------+
|  count|            64197|        64247|              64247|             64247|             64247|
|   mean|97.77009984890256|         null| 1993.6012420813422| 894.6413840334957|438943.70469516085|
| stddev|24.26994610142912|         null| 12.465629502278013|149.62669792791093|153760.65294972394|
|    min|             31.0|Adjoined flat|               1966|               553|          150000.0|
|    max|            249.0|      Type S2|               2016|              1160|         1205000.0|
+-------+-----------------+-------------+-------------------+------------------+------------------+



### Step 2: Exploratory Data Analysis and Data Cleansing

In [39]:
# consider NaN Treatment
# show null value count in each column
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|year|month|town|flat_type|block|street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+
|   0|    0|   0|        0|    0|          0|           0|            50|         0|                  0|              0|           0|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+



In [40]:
# filter and show rows with NaNs
df1 = df.filter(df.floor_area_sqm.isNull())
df1.show()

+----+-----+---------------+---------+-----+------------------+------------+--------------+----------+-------------------+---------------+------------+
|year|month|           town|flat_type|block|       street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|
+----+-----+---------------+---------+-----+------------------+------------+--------------+----------+-------------------+---------------+------------+
|2017|    3|     QUEENSTOWN|   4 ROOM|  161|       MEI LING ST|    19 TO 21|          null|  Improved|               1970|            628|    556000.0|
|2017|    3|      TOA PAYOH|   4 ROOM|  220|   LOR 8 TOA PAYOH|    16 TO 18|          null|  Improved|               1976|            698|    383000.0|
|2017|    3|      TOA PAYOH|   4 ROOM|  213|   LOR 8 TOA PAYOH|    16 TO 18|          null|  Improved|               1975|            683|    390000.0|
|2017|    3|      TOA PAYOH|   4 ROOM|  204|     TOA PAYOH NTH|    13 TO 15|          nu

In [41]:
#Type of flats that have null values
df1.select('flat_type').distinct().collect()

[Row(flat_type='4 ROOM')]

In [42]:
#The towns that have null values
df1.select('Town').distinct().collect()

[Row(Town='QUEENSTOWN'),
 Row(Town='BEDOK'),
 Row(Town='WOODLANDS'),
 Row(Town='BUKIT MERAH'),
 Row(Town='TOA PAYOH'),
 Row(Town='KALLANG/WHAMPOA'),
 Row(Town='MARINE PARADE'),
 Row(Town='CENTRAL AREA')]

In [43]:
#The flat models that have null values
df1.select('flat_model').distinct().collect()

[Row(flat_model='Improved')]

In [44]:
#setting up imputer for replacement of null values with mean
imputer = Imputer(
    inputCols=['floor_area_sqm'], 
    outputCols=["{}_imputed".format(c) for c in ['floor_area_sqm']]
    ).setStrategy("mean")

In [45]:
# Add imputation cols to df
df_cleaned = imputer.fit(df).transform(df)
df_cleaned.show()

+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+----------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|floor_area_sqm|    flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----------+---------+-----+-----------------+------------+--------------+--------------+-------------------+---------------+------------+----------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|          44.0|      Improved|               1979|            736|    232000.0|                  44.0|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|          67.0|New Generation|               1978|            727|    250000.0|                  67.0|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|          67.0|New Generation|               1980|            749

In [46]:
# consider NaN Treatment
# show null value count in each column
df_cleaned.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).show()

+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+----------------------+
|year|month|town|flat_type|block|street_name|storey_range|floor_area_sqm|flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+----------------------+
|   0|    0|   0|        0|    0|          0|           0|            50|         0|                  0|              0|           0|                     0|
+----+-----+----+---------+-----+-----------+------------+--------------+----------+-------------------+---------------+------------+----------------------+



In [47]:
df_cleaned = df_cleaned.drop('floor_area_sqm')

In [48]:
df_cleaned.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)



In [49]:
#Create new column "Price per sqm"

df3 = df_cleaned.withColumn("price_per_sqm", (df_cleaned.resale_price / df_cleaned.floor_area_sqm_imputed))
df3.show()

+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|     price_per_sqm|
+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+------------------+
|2017|    1|ANG MO KIO|   2 ROOM|  406|ANG MO KIO AVE 10|    10 TO 12|      Improved|               1979|            736|    232000.0|                  44.0| 5272.727272727273|
|2017|    1|ANG MO KIO|   3 ROOM|  108| ANG MO KIO AVE 4|    01 TO 03|New Generation|               1978|            727|    250000.0|                  67.0|3731.3432835820895|
|2017|    1|ANG MO KIO|   3 ROOM|  602| ANG MO KIO AVE 5|    01 TO 03|New Generation|               1980|          

In [50]:
df3.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- town: string (nullable = true)
 |-- flat_type: string (nullable = true)
 |-- block: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- storey_range: string (nullable = true)
 |-- flat_model: string (nullable = true)
 |-- lease_commence_date: integer (nullable = true)
 |-- remaining_lease: integer (nullable = true)
 |-- resale_price: double (nullable = true)
 |-- floor_area_sqm_imputed: double (nullable = true)
 |-- price_per_sqm: double (nullable = true)



In [51]:
popular_flat_df = df3.select('year', 'flat_type')
popular_flat_df = popular_flat_df.groupBy('year').count()
popular_flat_df.sort(asc('year')).show()

+----+-----+
|year|count|
+----+-----+
|2017|20500|
|2018|21561|
|2019|22186|
+----+-----+



In [52]:
#most popular type of flat for each year
df3.groupBy('year','flat_type').count().select(['year','flat_type','count']).sort(desc('count')).show(3)

+----+---------+-----+
|year|flat_type|count|
+----+---------+-----+
|2019|   4 ROOM| 9228|
|2018|   4 ROOM| 8769|
|2017|   4 ROOM| 8595|
+----+---------+-----+
only showing top 3 rows



In [53]:
resale_time_df = df3.select('year', 'month', 'resale_price')
resale_time_shown_df = resale_time_df.groupBy(('month'),(resale_time_df['year'])).mean('resale_price')
resale_time_shown_df = resale_time_shown_df.sort(desc('avg(resale_price)'))
resale_time_shown_df.show(36)

+-----+----+------------------+
|month|year| avg(resale_price)|
+-----+----+------------------+
|    3|2018|457069.77033492824|
|    9|2017|452715.63333333336|
|   12|2017|449859.98738170345|
|    5|2018| 448849.0189655172|
|    6|2017| 448502.0360618203|
|    4|2018| 448189.2339499456|
|    2|2017| 447659.2294172063|
|    1|2018|446379.56623134325|
|    8|2017|445079.46530612244|
|    3|2017|445029.89694167103|
|    7|2018| 444896.9440724695|
|    5|2017| 443789.3212121212|
|   10|2017| 443583.7658263305|
|    2|2018|443147.78360101435|
|   11|2017|443040.18478787877|
|    6|2018|442732.18329113926|
|    8|2018| 441601.1750749879|
|    7|2017|438739.63033707865|
|    4|2017| 438629.2722349103|
|   11|2019| 434429.5716649215|
|    7|2019| 434089.4914285714|
|    9|2018| 433734.4224836272|
|    5|2019| 433599.3990338164|
|    3|2019| 433256.5438596491|
|    8|2019| 432779.7465194532|
|    9|2019| 432455.4672131148|
|    6|2019|431988.53768577497|
|   12|2019|431583.52705627703|
|    4|2

In [54]:
#bottom 5 towns based on average resale price
df3.groupBy('Town','flat_type').mean().select(['Town','flat_type','avg(resale_price)']).sort(asc('avg(resale_price)')).show(10)

+-------------+---------+------------------+
|         Town|flat_type| avg(resale_price)|
+-------------+---------+------------------+
|  BUKIT MERAH|   1 ROOM| 186181.6551724138|
|      GEYLANG|   2 ROOM|        202859.375|
|    TOA PAYOH|   2 ROOM|207133.48387096773|
|MARINE PARADE|   2 ROOM|          209000.0|
|    SERANGOON|   2 ROOM|          211478.0|
|   ANG MO KIO|   2 ROOM|216264.15094339623|
|    WOODLANDS|   2 ROOM|217269.33333333334|
|  JURONG WEST|   2 ROOM|218062.51162790696|
|        BEDOK|   2 ROOM|220144.68085106384|
|    SEMBAWANG|   2 ROOM|223402.43902439025|
+-------------+---------+------------------+
only showing top 10 rows



In [55]:
#top 5 towns based on average resale price
df3.groupBy('Town','flat_type').mean().select(['Town','flat_type',
                                                      'avg(resale_price)']).sort(desc('avg(resale_price)')).show(10)

+------------+----------------+-----------------+
|        Town|       flat_type|avg(resale_price)|
+------------+----------------+-----------------+
|  QUEENSTOWN|       EXECUTIVE|         993350.0|
| BUKIT TIMAH|       EXECUTIVE|951174.4761904762|
|CENTRAL AREA|          5 ROOM|      944515.4875|
|      BISHAN|MULTI-GENERATION|         914777.6|
|      BISHAN|       EXECUTIVE|899599.6033057851|
|    TAMPINES|MULTI-GENERATION|846222.2222222222|
|  QUEENSTOWN|          5 ROOM|836754.3557312253|
|    CLEMENTI|       EXECUTIVE|830979.0188679246|
|  ANG MO KIO|       EXECUTIVE|817382.6666666666|
| BUKIT TIMAH|          5 ROOM|815171.4782608695|
+------------+----------------+-----------------+
only showing top 10 rows



In [86]:
#Towns with the oldest flats
df3.groupBy('Town').mean().select(['Town','avg(remaining_lease)', 'avg(price_per_sqm)', 'avg(floor_area_sqm_imputed)']).sort(asc('avg(remaining_lease)')).show(5)

+-------------+--------------------+------------------+---------------------------+
|         Town|avg(remaining_lease)|avg(price_per_sqm)|avg(floor_area_sqm_imputed)|
+-------------+--------------------+------------------+---------------------------+
|MARINE PARADE|   672.8940568475452|  5976.20374608566|          85.24131963550985|
|      GEYLANG|   764.7574002574003| 5059.600640454359|          83.14356499356502|
|        BEDOK|   775.6859142607174| 4549.276523713413|          89.51111711842911|
|  BUKIT TIMAH|   778.6878306878307| 6419.156959117102|         110.78306878306879|
|   ANG MO KIO|   781.9516626671237| 4726.984564654567|          84.91909496057593|
+-------------+--------------------+------------------+---------------------------+
only showing top 5 rows



In [87]:
#Towns with the newest flats
df3.groupBy('Town').mean().select(['Town','avg(remaining_lease)', 'avg(price_per_sqm)', 'avg(floor_area_sqm_imputed)']).sort(desc('avg(remaining_lease)')).show(5)

+-------------+--------------------+------------------+---------------------------+
|         Town|avg(remaining_lease)|avg(price_per_sqm)|avg(floor_area_sqm_imputed)|
+-------------+--------------------+------------------+---------------------------+
|      PUNGGOL|  1092.5940692748568|4702.6199711518575|          97.12708696735609|
|     SENGKANG|  1050.2722334004025| 4399.344359673143|          99.77424547283702|
|    SEMBAWANG|  1005.6989737742304|3686.5537524548918|          103.7212086659065|
|BUKIT PANJANG|   963.4236577181208| 4125.453614235329|         103.97944630872483|
|    WOODLANDS|   933.2477947072975| 3560.995749274707|         106.17256818361047|
+-------------+--------------------+------------------+---------------------------+
only showing top 5 rows



In [58]:
df3.groupBy('Town','flat_type').mean().select(['Town','flat_type','avg(floor_area_sqm_imputed)', 'avg(remaining_lease)']).sort(desc('avg(remaining_lease)')).show(10)

+-------------+---------+---------------------------+--------------------+
|         Town|flat_type|avg(floor_area_sqm_imputed)|avg(remaining_lease)|
+-------------+---------+---------------------------+--------------------+
|    SEMBAWANG|   3 ROOM|          67.64406779661017|  1139.3389830508474|
|    PASIR RIS|   2 ROOM|                       47.0|              1136.0|
|      PUNGGOL|   3 ROOM|          68.37288135593221|  1133.6474576271187|
|BUKIT PANJANG|   2 ROOM|                       47.0|  1131.7407407407406|
|      PUNGGOL|   2 ROOM|          46.94871794871795|  1131.1153846153845|
|     SENGKANG|   3 ROOM|          67.69393939393939|   1130.360606060606|
|    SEMBAWANG|   2 ROOM|          45.48780487804878|   1130.341463414634|
|  JURONG EAST|   2 ROOM|         44.794117647058826|              1127.5|
|     SENGKANG|   2 ROOM|           46.9537037037037|  1126.3055555555557|
|     TAMPINES|   2 ROOM|          46.03333333333333|  1125.9333333333334|
+-------------+---------+

In [59]:
#Average floor area by m2 and Average Remaining Lease for each flat type
df3.groupBy('flat_type').mean().select(['flat_type','avg(floor_area_sqm_imputed)', 'avg(remaining_lease)']).sort(asc('avg(remaining_lease)')).show()

+----------------+---------------------------+--------------------+
|       flat_type|avg(floor_area_sqm_imputed)|avg(remaining_lease)|
+----------------+---------------------------+--------------------+
|          1 ROOM|                       31.0|   668.5172413793103|
|          3 ROOM|          68.29045480787735|   779.8618897940855|
|MULTI-GENERATION|          164.8181818181818|   815.0909090909091|
|       EXECUTIVE|         144.30160572644613|   895.2505320177984|
|          2 ROOM|         45.869423286180634|   913.0533188248096|
|          4 ROOM|          95.57121333455346|   936.4632972322503|
|          5 ROOM|         118.18541090726313|   936.5038326212616|
+----------------+---------------------------+--------------------+



In [60]:
#Remaining Lease vs Resale Price
df3.groupBy('remaining_lease').mean().select(['remaining_lease','avg(resale_price)']).sort(asc('remaining_lease')).show(5)
df3.groupBy('remaining_lease').mean().select(['remaining_lease','avg(resale_price)']).sort(desc('remaining_lease')).show(5)

+---------------+------------------+
|remaining_lease| avg(resale_price)|
+---------------+------------------+
|            553|          258700.0|
|            554|245166.66666666666|
|            555|          256000.0|
|            556|          246000.0|
|            557|          265000.0|
+---------------+------------------+
only showing top 5 rows

+---------------+-----------------+
|remaining_lease|avg(resale_price)|
+---------------+-----------------+
|           1160|         695000.0|
|           1159|         462500.0|
|           1158|         861000.0|
|           1157|         839000.0|
|           1156|957666.6666666666|
+---------------+-----------------+
only showing top 5 rows



In [61]:
#Comparison of 3room, 4room and 5room resale prices after 3 years

#3-room
df_room = df3.groupBy('flat_type', 'year', 'month').mean().select(['flat_type','year','month','avg(resale_price)'])
df_room.filter((df_room['flat_type']=='3 ROOM') & (df_room['year']=='2017') & (df_room['month']=='1')).show()
df_room.filter((df_room['flat_type']=='3 ROOM') & (df_room['year']=='2019') & (df_room['month']=='12')).show()

#4-room
df_room = df3.groupBy('flat_type', 'year', 'month').mean().select(['flat_type','year','month','avg(resale_price)'])
df_room.filter((df_room['flat_type']=='4 ROOM') & (df_room['year']=='2017') & (df_room['month']=='1')).show()
df_room.filter((df_room['flat_type']=='4 ROOM') & (df_room['year']=='2019') & (df_room['month']=='12')).show()

#5-room
df_room = df3.groupBy('flat_type', 'year', 'month').mean().select(['flat_type','year','month','avg(resale_price)'])
df_room.filter((df_room['flat_type']=='5 ROOM') & (df_room['year']=='2017') & (df_room['month']=='1')).show()
df_room.filter((df_room['flat_type']=='5 ROOM') & (df_room['year']=='2019') & (df_room['month']=='12')).show()

+---------+----+-----+-----------------+
|flat_type|year|month|avg(resale_price)|
+---------+----+-----+-----------------+
|   3 ROOM|2017|    1| 318483.656626506|
+---------+----+-----+-----------------+

+---------+----+-----+------------------+
|flat_type|year|month| avg(resale_price)|
+---------+----+-----+------------------+
|   3 ROOM|2019|   12|301084.65684210527|
+---------+----+-----+------------------+

+---------+----+-----+------------------+
|flat_type|year|month| avg(resale_price)|
+---------+----+-----+------------------+
|   4 ROOM|2017|    1|427766.58585858584|
+---------+----+-----+------------------+

+---------+----+-----+-----------------+
|flat_type|year|month|avg(resale_price)|
+---------+----+-----+-----------------+
|   4 ROOM|2019|   12| 433873.908496732|
+---------+----+-----+-----------------+

+---------+----+-----+-----------------+
|flat_type|year|month|avg(resale_price)|
+---------+----+-----+-----------------+
|   5 ROOM|2017|    1|  516695.73046875|
+-

### Step 3: Data Wrangling and Transformation

In [62]:
strings_used = ['town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model']

stage_string = [StringIndexer(inputCol= c, outputCol= c+"_string_encoded") for c in strings_used]
stage_one_hot = [OneHotEncoder(inputCol= c+"_string_encoded", outputCol= c+ "_one_hot") for c in strings_used]

ppl = Pipeline(stages= stage_string + stage_one_hot)
pyspark_index_ohe = ppl.fit(df3).transform(df3)
pyspark_index_ohe.show(5, truncate=False)

+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+------------------+-------------------+------------------------+--------------------+--------------------------+---------------------------+-------------------------+--------------+-----------------+------------------+-------------------+--------------------+------------------+
|year|month|town      |flat_type|block|street_name      |storey_range|flat_model    |lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|price_per_sqm     |town_string_encoded|flat_type_string_encoded|block_string_encoded|street_name_string_encoded|storey_range_string_encoded|flat_model_string_encoded|town_one_hot  |flat_type_one_hot|block_one_hot     |street_name_one_hot|storey_range_one_hot|flat_model_one_hot|
+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+-------

In [63]:
#Consolidating x columns
# specify the input and output columns of the vector assembler
assembler = VectorAssembler(inputCols=['year',
                                       'month',
                                       'lease_commence_date',
                                       'remaining_lease',
                                       'floor_area_sqm_imputed', 
                                       'price_per_sqm',
                                       'resale_price'],
                           outputCol='Xcols')
# transform the data
final_data = assembler.transform(pyspark_index_ohe)

# view the transformed vector
final_data.select('Xcols').show()

+--------------------+
|               Xcols|
+--------------------+
|[2017.0,1.0,1979....|
|[2017.0,1.0,1978....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1981....|
|[2017.0,1.0,1979....|
|[2017.0,1.0,1976....|
|[2017.0,1.0,1979....|
|[2017.0,1.0,1979....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1977....|
|[2017.0,1.0,1977....|
|[2017.0,1.0,1977....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1978....|
|[2017.0,1.0,1979....|
|[2017.0,1.0,1980....|
|[2017.0,1.0,1979....|
|[2017.0,1.0,1980....|
+--------------------+
only showing top 20 rows



In [64]:
#Standard Scaler
sScaler = StandardScaler(withMean=True, withStd=True, inputCol="Xcols", outputCol="Xcols_sscaled")

final_data = sScaler.fit(final_data).transform(final_data)
final_data.show()

+----+-----+----------+---------+-----+-----------------+------------+--------------+-------------------+---------------+------------+----------------------+------------------+-------------------+------------------------+--------------------+--------------------------+---------------------------+-------------------------+--------------+-----------------+------------------+-------------------+--------------------+------------------+--------------------+--------------------+
|year|month|      town|flat_type|block|      street_name|storey_range|    flat_model|lease_commence_date|remaining_lease|resale_price|floor_area_sqm_imputed|     price_per_sqm|town_string_encoded|flat_type_string_encoded|block_string_encoded|street_name_string_encoded|storey_range_string_encoded|flat_model_string_encoded|  town_one_hot|flat_type_one_hot|     block_one_hot|street_name_one_hot|storey_range_one_hot|flat_model_one_hot|               Xcols|       Xcols_sscaled|
+----+-----+----------+---------+-----+-----

### Step 4: Machine Learning Modelling

In [65]:
# how to we train or test our models?
splits = final_data.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [66]:
# use code to show number of rows and columns,
# as well as a sample of 10 rows before heading into Machine Learning Modelling
#Train dataset
print('Shape of the dataset: ', (train_df.count(), len(train_df.columns)))
#Test dataset
print('Shape of the dataset: ', (test_df.count(), len(test_df.columns)))

Shape of the dataset:  (44731, 27)
Shape of the dataset:  (19516, 27)


In [88]:
train_data = train_df.select("Xcols_sscaled","resale_price")
train_data.show(10)

+--------------------+------------+
|       Xcols_sscaled|resale_price|
+--------------------+------------+
|[-1.2596662517206...|    275000.0|
|[-1.2596662517206...|    325000.0|
|[-1.2596662517206...|    285000.0|
|[-1.2596662517206...|    297000.0|
|[-1.2596662517206...|    298000.0|
|[-1.2596662517206...|    295000.0|
|[-1.2596662517206...|    295000.0|
|[-1.2596662517206...|    321000.0|
|[-1.2596662517206...|    335000.0|
|[-1.2596662517206...|    330000.0|
+--------------------+------------+
only showing top 10 rows



In [89]:
test_data = test_df.select("Xcols_sscaled","resale_price")
test_data.show(10)

+--------------------+------------+
|       Xcols_sscaled|resale_price|
+--------------------+------------+
|[-1.2596662517206...|    232000.0|
|[-1.2596662517206...|    250000.0|
|[-1.2596662517206...|    330000.0|
|[-1.2596662517206...|    312000.0|
|[-1.2596662517206...|    373000.0|
|[-1.2596662517206...|    338000.0|
|[-1.2596662517206...|    306000.0|
|[-1.2596662517206...|    265000.0|
|[-1.2596662517206...|    328000.0|
|[-1.2596662517206...|    301000.0|
+--------------------+------------+
only showing top 10 rows



### Step 5: Model Evaluation and Selection

In [69]:
regressor=LinearRegression(featuresCol="Xcols_sscaled", labelCol='resale_price')
regressor=regressor.fit(train_data)

In [70]:
type(regressor)

pyspark.ml.regression.LinearRegressionModel

In [71]:
# Coefficients
regressor.coefficients

DenseVector([0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 153760.6529])

In [72]:
# Intercept
regressor.intercept

438943.7046951623

In [73]:
# first row values of Xcols_sscaled
train_data.take(1)

[Row(Xcols_sscaled=DenseVector([-1.2597, -1.7708, -1.0109, -0.9266, -1.2271, -0.3943, -1.0662]), resale_price=275000.0)]

In [74]:
train_pred_results=regressor.evaluate(train_data)

In [75]:
train_pred_results.predictions.show()

+--------------------+------------+------------------+
|       Xcols_sscaled|resale_price|        prediction|
+--------------------+------------+------------------+
|[-1.2596662517206...|    275000.0| 274999.9999991242|
|[-1.2596662517206...|    325000.0| 324999.9999995442|
|[-1.2596662517206...|    285000.0|284999.99999963713|
|[-1.2596662517206...|    297000.0|296999.99999988324|
|[-1.2596662517206...|    298000.0|297999.99999915797|
|[-1.2596662517206...|    295000.0| 295000.0000001223|
|[-1.2596662517206...|    295000.0| 295000.0000001223|
|[-1.2596662517206...|    321000.0| 320999.9999998767|
|[-1.2596662517206...|    335000.0|334999.99999988603|
|[-1.2596662517206...|    330000.0|329999.99999916623|
|[-1.2596662517206...|    325000.0| 324999.9999991752|
|[-1.2596662517206...|    280000.0|279999.99999985803|
|[-1.2596662517206...|    285000.0|284999.99999986193|
|[-1.2596662517206...|    390000.0|389999.99999975634|
|[-1.2596662517206...|    288500.0| 288499.9999992589|
|[-1.25966

In [76]:
test_pred_results=regressor.evaluate(test_data)

In [77]:
test_pred_results.predictions.show()

+--------------------+------------+------------------+
|       Xcols_sscaled|resale_price|        prediction|
+--------------------+------------+------------------+
|[-1.2596662517206...|    232000.0| 231999.9999997096|
|[-1.2596662517206...|    250000.0| 249999.9999999627|
|[-1.2596662517206...|    330000.0| 329999.9999996343|
|[-1.2596662517206...|    312000.0| 312000.0000003723|
|[-1.2596662517206...|    373000.0|372999.99999954633|
|[-1.2596662517206...|    338000.0| 337999.9999991723|
|[-1.2596662517206...|    306000.0| 305999.9999991603|
|[-1.2596662517206...|    265000.0|  264999.999999241|
|[-1.2596662517206...|    328000.0|327999.99999977916|
|[-1.2596662517206...|    301000.0| 301000.0000001107|
|[-1.2596662517206...|    425888.0| 425887.9999992462|
|[-1.2596662517206...|    425888.0| 425887.9999992462|
|[-1.2596662517206...|    425888.0| 425887.9999992462|
|[-1.2596662517206...|    430000.0| 429999.9999998594|
|[-1.2596662517206...|    560000.0| 560000.0000001364|
|[-1.25966

In [78]:
train_pred_results.meanAbsoluteError, train_pred_results.meanSquaredError, train_pred_results.r2

(3.779122508073815e-07, 2.049567361733361e-13, 1.0)

In [79]:
test_pred_results.meanAbsoluteError, test_pred_results.meanSquaredError, test_pred_results.r2

(3.756353910115257e-07, 2.0208333014798127e-13, 1.0)

### Step 7: Report

# 7.1 Problem Statement Formulation <a id="part7.1"></a>
[Back to top](#top)

First, before loading the data, I will import the packages required to accomplish the various tasks in this report. The packages are split into 8 parts that are Basic, Functions, Types, Matplotlib, Imputation, Categorical Encoding, Feature Scaling and Machine Learning.

I will load the housing dataset using the `spark.read.csv` function. Then, I will use the `printSchema` function to find out the data types of the variables present in the dataset. There are a total of 12 variables present in the datset, with 4 of them being `integer` type, 2 of them `double` type and the remaining 6 being of `string`type.

I will then use the `show` function to see the preview of 20 records present in the dataset, followed by the `describe` function to find out the basic information of each variable.
Over the years, the use of Machine Learning has proved to be an effective and essential metric to discover patterns, create models and make predictions. It has become more prominent in various sectors of work such as Medical Science, Information Technology, Sports and even Housing. Housing utilises and benefits greatly through the use of Machine Learning. Housing details is known for its incorporation of Data to help improve the resale value of a house, be it regarding the location, floor area, type of house etc.

Hence, as data plays a big part in this field, I wanted to see if the data compiled can accurately predict the resale price for each house using the dataset provided for this Assignment that consists of Housing Transactions data compiled from the year 2017 2019. I will be using Regression models to showcase the results that I have obtained.


# 7.2 Exploratory Data Analysis and Data Cleansing <a id="part7.2"></a>
[Back to top](#top)

Firstly, the null values in the dataset need to be dealt with using either Dropping, Replacing or Imputation methods. However, to do that, we need to find out which columns have null values present in them. Thus, we will use the following code : 
` select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()`

This code allows us to see the count of null values in each column in the dataset. After executing this, it can be observed that only the `floor_area_sqm` column has null values with 50.  We can then see the rows with null values present in them using the code: 
` filter(df.floor_area_sqm.isNull())`

From this, the null values present in the dataset belong to transactions involving 4-room flats of the Improved model type in a total of 8 different locations.
As these values are of immense values to the dataset, they cannot be dropped, nor replaced by random values. Thus, we need to use the Imputation method such that the null values are removed and instead replaced with the mean values in a new column called `floor_area_sqm_imputed`. We then remove the `floor_area_sqm` column as it is no longer needed.

We can then create new calculated columns such as `price_per_sqm` that divides the resale price by the floor area of a given house to see how much each square meter is really worth.

Then, we can proceed to conduct the Exploratory Data Analysis(EDA) for this report using functions such as `filter`, `groupBy`,  `sort` and  `select`.  The key takeaways from the EDA conducted is as follows:
-	The number of housing transactions increase year by year.
-	4-room flats are the most popular type of flats bought for all the 3 years.
-	Older flats generally cost more than newer flats despite being smaller in size
-	Majority of newer flats that are being built are either 2,4 or 5 room flats
-	The resale prices of 4-room and 5-room flats have appreciated after 3 years but the resale prices of 3-room flats has depreciated after 3 years


# 7.3 Data Wrangling and Transformation <a id="part7.3"></a>
[Back to top](#top)

Encoding Categorical Data is another essential facet of Machine Learning. Categorical variables are essentially variables that are either discrete and uncontinuous. As our data currently contains multiple categorical variable that the Regression model cannot use, we need to encode the Categorical Data. There are multiple variations of Categorical Data Encoding. The one we are going to use goes by the name of One Hot Encoding which represent a categorical variable as a group of binary variables, where each binary variable represents one category. The binary variable indicates whether the category is present in an observation (1) or not (0).
However, first, I conducted String indexing so that the string values in columns like ` 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'flat_model'` are encoded to the column label indices. Then I conducted One Hot Encoding by first creating the one hot encoder, then adding it to the dataframe using the `Pipeline` feature.

Then, I consolidated the X columns using the VectorAssemble feature. VectorAssembler allows us to convert all the numerical features into 1 single vector that is to be used by the Machine Learning Model.
For this assignment, we will use the Standardisation technique to conduct feature scaling. Standardisation refers to another scaling technique where the values are centered around the mean with a unit standard deviation. This results in the mean of the attribute becoming 0 and the resultant distribution in having a unit standard deviation. First, we will set up the StandardScaler before fitting the scaler to the dataset. Then, we will transform the dataset.


# 7.4 Machine Learning Modelling <a id="part7.4"></a>
[Back to top](#top)

As we have now successfully conducted various Data Wrangling and Data Cleaning and Transformation Techniques, we can now build the Machine Learning Model. As our prediction problem is a Regression problem, a Regression model needs to be built. Hence, a Linear Regression Model is to be built.

However, before we build the Linear Regression Model, we need to first conduct the Train Test Split to get our Train and Test Datasets and then state the number of rows and columns in our final dataset. This will help show that the predictions are not trivial nor unrealistic.

The train set has a total of 44731 rows and 27 columns while the test dataset has a total of 19516 rows and 27 columns. A preview of both the train and test datasets, consisting of 10 rows is then displayed.


# 7.5 Model Evaluation and Metrics <a id="part7.5"></a>
[Back to top](#top)

Linear regression is defined as a machine learning model in which the model finds the best fit linear line between the independent and dependent variables in which the aim is to find the best fit linear line and the optimal values of intercept and coefficients such that errors are minimized. For the purpose of this analysis, the goal is to use the X variables to predict the `resale_price` ( target variable).

In order to evaluate the model performance, a range of metrics can be used, but in this report, we will focus on only 3:
- Mean Square Error (MSE): The MSE is the average squared difference between the predicted values and the actual values in a dataset. The lower the MSE, the better said model fits a dataset.
- Mean Absolute Error (MAE): The MAE is the absolute average distance between the predicted values and the actual values in the dataset. The lower the MAE, the better the model fits the dataset.
- R-Square: R-square represents the percentage of the response variable variation that is explained by a linear model. The closer to 1.0 the value of R-square, the more indicative of a better fit and model performance.

Upon seeing the predictions made by the Linear Regression model, it can be observed clearly that the prediction values are nearly equivalent to their respective original resale price values. This is backed up by the R-Square values which has a value of 1.0 for both the train and the test datasets. 


# 7.6 Summary and Further Improvements <a id="part7.6"></a>
[Back to top](#top)

In this report, we conducted Data Cleansing, Data Exploration, Data Wrangling, Transformation and Feature Scaling techniques. We also created a Linear Regression model that returned us with the MSE, MAE and R-Square values. The `MSE` metric for the Tests set achieved a really good score of `2.02 ` and the `MAE` metric also attained a good score of `3.76 ` which means the dataset fit the `Linear Regression` model really well. The `R2` metric achieved an amazing score of `1.0 `, highlighting that the `Linear Regression model` performed really well.

At the end of this report, it can be seen that the model fits the dataset accurately, which is too good to be true in a real case scenario as it is impossible that all the predicted values are deemed to be near perfect. This is one aspect that needs to be worked on. Moreover, I only used 1 model, `Linear Regression Model`, when I could have used more, to get a variety of models to choose from to see which best fits the dataset.


### "Unlisted" Youtube Link to Video Presentation

In [80]:
# insert your link in this cell, you are allowed to comment it out
# youtube link: https://youtu.be/sF66102uTCE